Import libraries

In [1]:
import numpy as np
import pandas as pd
import itertools
from Bio import SeqIO
from tensorflow import keras
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

Load datasets

In [2]:
df_melanogaster = pd.DataFrame(
    [(str(el.seq), 0 if el.id == "Nucleosome" else 1) for el in SeqIO.parse("data/nucleosomes_vs_linkers_melanogaster.fas", "fasta")], 
    columns=['sequence', 'label'],
)
df_melanogaster.name = 'melanogaster'

df_elegans = pd.DataFrame(
    [(str(el.seq), 0 if el.id == "Nucleosome" else 1) for el in SeqIO.parse("data/nucleosomes_vs_linkers_elegans.fas", "fasta")], 
    columns=['sequence', 'label']
)
df_elegans.name = 'elegans'

df_sapiens = pd.DataFrame(
    [(str(el.seq), 0 if el.id == "Nucleosome" else 1) for el in SeqIO.parse("data/nucleosomes_vs_linkers_sapiens.fas", "fasta")], 
    columns=['sequence', 'label']
)
df_sapiens.name = 'sapiens'

Visualize dataset

In [3]:
df_melanogaster

,sequence,label
0,GGCCCCCCTGAACCCCAAGGCTAACCGCGAGAAGATGACCCAGATC...,0
1,TCAAATGAACAAAGGCAAAGGAGTCCACAGACCAGGAGATAAAACC...,0
2,TTCGCTAGCAAATTATTAGGCAATTAAATTAATAAATGAATTGTAT...,0
3,TAGTTTATAATTAATTCATATACACAAAAGAAAAAGAAAATTAAAT...,0
4,GATAGTTGTATTAAGTATTAGTATTAATATGTTTAATTTGTCTGTA...,0
...,...,...
5745,AAAACGCTGTGGCACATTTCTCTGATAAGCAAATTCTCTTCCTGAG...,1
5746,AAAGCGGCGCGAAAAAAAGGTTACTTCTGTTAACTGCTGTGTGTCA...,1
5747,CTTTACCCTGCATACTACCAACCATCCGGATGACAGACAACACGGT...,1
5748,CGCATTTCGTGGGTATGGCTAATGATCGCGTATGTAGAGTATTCAC...,1


Create W for k-mers representatin

In [4]:
k = 2;
W = [combination[0] + combination[1] for combination in itertools.product(["A", "C", "G", "T"], repeat=2)]
W

['AA',
 'AC',
 'AG',
 'AT',
 'CA',
 'CC',
 'CG',
 'CT',
 'GA',
 'GC',
 'GG',
 'GT',
 'TA',
 'TC',
 'TG',
 'TT']

Define the one-hot encoder

In [5]:
def one_hot_encoder(sequence):
    mapping = dict(zip("ACGT", range(4)))
    one_hot = np.asarray([[0 for _ in range(0, len(sequence))] for base in "ACGT"])
    for i in range(0, len(sequence)):
        one_hot[mapping[sequence[i]]][i] = 1
    return [i for l in one_hot for i in l]

In [6]:
results = {}

for dataframe in [df_melanogaster, df_elegans, df_sapiens]:

    results[dataframe.name] = {}

    # Represent the sequence in k-mers

    x_dataset_k_mers = []
    for sequence in dataframe["sequence"]:
        k_mers = [0 for _ in range(0, 4 ** k)]
        for i in range(0, len(sequence) - 1):
            k_mers[W.index(sequence[i:i+2])] += 1
        l = len(sequence) - k + 1;
        x_dataset_k_mers.append([n / l for n in k_mers])

    x_dataset_k_mers = np.asarray(x_dataset_k_mers)


    # Represent the sequence in one-hot encoding

    x_dataset_one_hot = np.asarray([one_hot_encoder(sequence) for sequence in dataframe["sequence"]])


    # Create the y for the dataset

    y_dataset = np.asarray(dataframe["label"])
    y_dataset_categorical = keras.utils.to_categorical(y_dataset, 2)


    # Define the models

    k_mers_model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(len(x_dataset_k_mers[0]),)),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(2, activation="sigmoid"),
    ])

    k_mers_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


    one_hot_model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(len(x_dataset_one_hot[0]),)),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(2, activation="sigmoid"),
    ])

    one_hot_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


    # 10-fold cross-validation

    n_fold = 10
    kf = KFold(n_splits=n_fold, shuffle=True)
    k_mers_accuracies = []
    one_hot_accuracies = []
    for train_index, test_index in kf.split(x_dataset_k_mers):
        y_train = y_dataset_categorical[train_index]
        y_test = y_dataset[test_index]

        k_mers_x_train, k_mers_x_test = x_dataset_k_mers[train_index], x_dataset_k_mers[test_index]
        k_mers_model.fit(k_mers_x_train, y_train, epochs=64, batch_size=64, validation_split=0.2, shuffle=True, verbose=False)
        k_mers_model_y_predicted_probabilities = k_mers_model.predict(k_mers_x_test)
        k_mers_model_y_predicted = np.argmax(k_mers_model_y_predicted_probabilities, axis=1)
        k_mers_accuracies.append(classification_report(y_test, k_mers_model_y_predicted, output_dict=True)["accuracy"])
        
        one_hot_x_train, one_hot_x_test = x_dataset_one_hot[train_index], x_dataset_one_hot[test_index]
        one_hot_model.fit(one_hot_x_train, y_train, epochs=64, batch_size=64, validation_split=0.2, shuffle=True, verbose=False)
        one_hot_model_y_predicted_probabilities = one_hot_model.predict(one_hot_x_test)
        one_hot_model_y_predicted = np.argmax(one_hot_model_y_predicted_probabilities, axis=1)
        one_hot_accuracies.append(classification_report(y_test, one_hot_model_y_predicted, output_dict=True)["accuracy"])


    # Compute the mean accuracy

    results[dataframe.name]["k_mers_accuracy"] = sum(k_mers_accuracies) / n_fold
    results[dataframe.name]["one_hot_accuracy"] = sum(one_hot_accuracies) / n_fold

results

{'elegans': {'k_mers_accuracy': 0.8546914557552855,
  'one_hot_accuracy': 0.9343132715473141},
 'melanogaster': {'k_mers_accuracy': 0.7916521739130434,
  'one_hot_accuracy': 0.8928695652173912},
 'sapiens': {'k_mers_accuracy': 0.8344849168203492,
  'one_hot_accuracy': 0.8839321376358059}}